In [19]:
## Clasificacion-de-sensores-espaciales-con-Machine-Learning-KNN
#Alvaro Daniel Hernandez

#-------------
#Ingresar ubicación del archivo
archivo_en_drive = 'C:\\Users\\jamer\\Downloads\\datos_sensores.db'
#Importar las librerias necesarias

import sqlite3 #para manejar archivo sql lite
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Conectar a la base de datos
conexion = sqlite3.connect(archivo_en_drive)

# Consulta SQL para obtener los datos
#En este caso solo se tomaran los datos con etiqueta; porque se usaran para entrenar y para test
query = """
SELECT datos_basicos.id,datos_basicos.tamaño, datos_basicos.categoria,
orion.sensor_orion, vega.sensor_vega,
polaris.sensor_polaris, antares.sensor_antares,
CASE
 WHEN clasificacion.etiqueta='Positivo' THEN 1
 WHEN clasificacion.etiqueta='Negativo' THEN 0
END AS etiqueta_numerica
FROM datos_basicos
LEFT JOIN orion ON orion.id = datos_basicos.id
LEFT JOIN vega ON vega.id = datos_basicos.id
LEFT JOIN polaris ON polaris.id = datos_basicos.id
LEFT JOIN antares ON antares.id = datos_basicos.id
LEFT JOIN clasificacion ON clasificacion.id = datos_basicos.id
WHERE etiqueta_numerica IS NOT NULL
"""

# Leer los datos
dataset = pd.read_sql_query(query, conexion)

# Dividir los datos en conjuntos de entrenamiento y prueba
X = dataset.drop(['etiqueta_numerica', 'id'], axis=1)
y = dataset['etiqueta_numerica']
#Se toma el 20% de los datos a evaluar, para hacer pruebas 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Consulta SQL para obtener los datos faltantes
#En este caso como son los datos faltantes se toman los datos sin etiqueta, estos son los datos a predecir
faltante = """
SELECT datos_basicos.id,datos_basicos.tamaño, datos_basicos.categoria,
orion.sensor_orion, vega.sensor_vega,
polaris.sensor_polaris, antares.sensor_antares,
CASE
 WHEN clasificacion.etiqueta='Positivo' THEN 1
 WHEN clasificacion.etiqueta='Negativo' THEN 0
END AS etiqueta_numerica
FROM datos_basicos
LEFT JOIN orion ON orion.id = datos_basicos.id
LEFT JOIN vega ON vega.id = datos_basicos.id
LEFT JOIN polaris ON polaris.id = datos_basicos.id
LEFT JOIN antares ON antares.id = datos_basicos.id
LEFT JOIN clasificacion ON clasificacion.id = datos_basicos.id
WHERE etiqueta_numerica IS NULL
"""

# Leer los datos faltantes
data_faltante = pd.read_sql_query(faltante, conexion)

# Eliminar la columna etiqueta_numerica de los datos faltantes e id porque no hacen parte de x 
X_faltante = data_faltante.drop(['etiqueta_numerica', 'id'], axis=1)

# CREACION DEL MODELO KNN
knn_modelo = KNeighborsClassifier(n_neighbors=7,algorithm='auto', weights='distance')  # Puedes ajustar el valor de n_neighbors según tus necesidades
knn_modelo.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predicciones = knn_modelo.predict(X_test)
#Evaluar el modelo
#Accuracy
accuracy = accuracy_score(y_test, predicciones)
print(f'Acuraccy en conjunto de prueba: {accuracy}')
# Reporte de Clasificación 
Reporte_clasificacion=classification_report(y_test, predicciones)
print(f'El reporte de Clasificación es:')
print(Reporte_clasificacion)
#Matriz de confusion
matriz_confusion= confusion_matrix(y_test, predicciones)
print(f'La matriz de confusión es:')
print(matriz_confusion)




Acuraccy en conjunto de prueba: 0.9446333333333333
El reporte de Clasificación es:
              precision    recall  f1-score   support

           0       0.95      0.94      0.94     30142
           1       0.94      0.95      0.94     29858

    accuracy                           0.94     60000
   macro avg       0.94      0.94      0.94     60000
weighted avg       0.94      0.94      0.94     60000

La matriz de confusión es:
[[28267  1875]
 [ 1447 28411]]


Se evalua el modelo a los datos faltantes

In [20]:
# Realizar predicciones en los datos faltantes
prediccion_faltante = knn_modelo.predict(X_faltante)

# Mapear los valores de 0 y 1 a 'Negativo' y 'Positivo' respectivamente
prediccion_faltante_str = pd.Series(prediccion_faltante).map({0: 'Negativo', 1: 'Positivo'})

#Datatframe
# Crear un DataFrame con los IDs y las predicciones
resultados_prediccion = pd.DataFrame({'id': data_faltante['id'].astype(int), 'etiqueta': prediccion_faltante_str})

# Guardar el DataFrame en un archivo CSV
resultados_prediccion.to_csv('C:\\Users\\jamer\\Desktop\\Taller2_ayc\\output\\predicciones.csv', index=False)
